In [7]:
from dotenv import load_dotenv
from IPython.display import display, Markdown, JSON
load_dotenv()

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langchain_aws import ChatBedrock

# 1. CONFIGURACIÓN PARA DEEPSEEK-R1 (Razonamiento Complejo)
# Ideal para agentes que necesitan planificar pasos lógicos.
# llm = ChatBedrock(
#     model_id="us.deepseek.r1-v1:0",  # ID oficial validado
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.6, # DeepSeek recomienda 0.6 para razonamiento
#         "max_tokens": 8192,  # Recomendado para no degradar calidad del CoT
#         "top_p": 0.95,
#     }
# )


# llm = ChatBedrock(
#     model_id="us.deepseek.v3-v1:0", # Prueba este primero
#     region_name="us-east-1",        # O us-west-2
#     model_kwargs={
#         "temperature": 0.7,
#         "max_tokens": 4096
#     }
# )
from langchain_aws import ChatBedrock
llm = ChatBedrock(
model_id="us.meta.llama4-scout-17b-instruct-v1:0",  # Nota el prefijo "us."
# model_id="cohere.command-r-plus-v1:0",
region_name="us-east-1",
model_kwargs={
"temperature": 0.5,
"max_tokens": 2048,
"top_p": 0.9,
}
)


# llm = ChatBedrock(
#     model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

# llm = ChatBedrock(
#     model_id="amazon.nova-lite-v1:0",  # Nota el prefijo "us."
#     region_name="us-east-1",
#     model_kwargs={
#         "temperature": 0.5,
#         "max_tokens": 2048,
#         "top_p": 0.9,
#     }
# )

In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model=llm,
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="amazon.nova-lite-v1:0",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [4]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis is clear with temperatures ranging from a high of 120C to a low of -100C. There are 100,000 cheese miners living in Lunapolis, and they are unhappy with the new president, leading to a potential strike by the cheese miners' union.", additional_kwargs={}, response_metadata={}, id='d39df019-36fd-4156-a6f5-524d58fcb932'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='d6e4ce5a-b4bf-4f01-9172-2dc54cf755c3'),
              AIMessage(content='\n\nA crisis on the moon already! As the new president of Lunapolis, my primary goal is to address the concerns of the 100,000 cheese miners and prevent a potential strike. Here\'s my response:\n\n**Immediate Response**\n\nI would schedule an emergency meeting with the leaders of th

In [8]:
JSON(response)

<IPython.core.display.JSON object>

In [5]:
display(Markdown(response["messages"][0].content))

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis is clear with temperatures ranging from a high of 120C to a low of -100C. There are 100,000 cheese miners living in Lunapolis, and they are unhappy with the new president, leading to a potential strike by the cheese miners' union.

## Trim/delete messages

In [9]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [10]:
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [11]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='b077698a-8dde-4651-bc42-67c4966a2bc4'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='f3fade69-a1d7-4d9d-839a-8545487b3883'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='ec28bd5c-1508-41a7-a1f4-f75f1181d7bc'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='13258006-01bd-4ced-b1cb-1d9616311b38'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='66911fad-8f19-4cdb-84f1-25f135319f69'),
              AIMessage(content="\n\nIf the device is overheating or extremely cold, it might not turn on. Try moving it to a room-temperature environment and letting it a

In [13]:
JSON(response)

<IPython.core.display.JSON object>

In [12]:
display(Markdown(response["messages"][-1].content))



If the device is overheating or extremely cold, it might not turn on. Try moving it to a room-temperature environment and letting it acclimate for a bit.

Also, can you tell me what kind of device you're trying to turn on (e.g. phone, laptop, tablet, etc.)? That might help me provide more specific troubleshooting steps.